In [2]:
import requests
import os
import json
from zipfile import ZipFile
import pandas as pd
import geopandas as gpd
from datetime import datetime, timezone, date
from shapely.validation import make_valid
from arcgis.geometry import Geometry

Change this

In [3]:
# May need updating
location = "work"
item_url_file = "ItemURL_ArcGIS_6monthUpdates.csv" # Replace with "ItemURL_ArcGIS_YearlyUpdates.csv" for yearly updates 

Should not need updating

In [4]:
if location == "home":
    folder_location = "C:/Users/Lara/Work/DataDownloads/"
    
if location == "work":
    folder_location = "O:/Data_team/GIS_data_downloads/"

download_location = f"G:/GIS_Data/External/Open_Source/"
temp_download_location = f"{folder_location}testDataTemp/" # check if still need this
lookup_location = f"{folder_location}Lookups/"
boundary_file = f"{folder_location}CountyBoundary.shp"


In [6]:
# Wiltshire boundary for creating Wilts datasets
boundary_gdf = gpd.read_file(boundary_file)

In [9]:
# Read the csv file 
item_url_df = pd.read_csv(lookup_location + item_url_file)
item_url_df.tail(8)

,Dataset,Owner,URL
2,Priority Habitats Inventory (England),NE,https://services.arcgis.com/JJzESW51TqeY9uat/a...
3,Ramsar (England),NE,https://services.arcgis.com/JJzESW51TqeY9uat/a...
4,Sites of Special Scientific Interest (England),NE,https://services.arcgis.com/JJzESW51TqeY9uat/a...
5,Sites of Special Scientific Interest Units (En...,NE,https://services.arcgis.com/JJzESW51TqeY9uat/a...
6,Special Areas of Conservation (England),NE,https://services.arcgis.com/JJzESW51TqeY9uat/a...
7,Special Protection Areas (England),NE,https://services.arcgis.com/JJzESW51TqeY9uat/a...
8,SSSI Impact Risk Zones (England),NE,https://services.arcgis.com/JJzESW51TqeY9uat/a...
9,Ancient Tree Inventory (ATI),Woodland Trust,https://services-eu1.arcgis.com/WIfgdJeDbrZU1c...


In [10]:
#Loop through each dataset name in lookup
for url in item_url_df.tail(7).URL:
    print(url)
    # Get url that contains details for dataset
    url_details = url.replace("/query?", "?f=json")
    
    # Get dataset name and source from lookup
    dataset = item_url_df[item_url_df.URL==url]['Dataset'].item() 
    dataset = dataset.replace(":", "-")
    source = item_url_df[item_url_df.URL==url]['Owner'].item()
    
    def fetch_page(offset, limit=2000):
        """Fetch a single page of features from ArcGIS FeatureServer."""
        params = {
            "where": "1=1",
            "outFields": "*",
            "returnGeometry": "true",
            "f": "json",
            "outSR": 27700,  # EPSG:27700
            "resultOffset": offset,
            "resultRecordCount": limit
        }
        return requests.get(url, params=params).json()

    records = []
    geoms = []

    offset = 0
    page_size = 2000

    # Keeps running while there is still data to download
    while True:
        print(f"Downloading features {offset} → {offset + page_size} …")
        try:
            data = fetch_page(offset, page_size)
        except Exception as error:
            print(f"Request error occured for {dataset}:", error)   
        feats = data.get("features", [])
        if not feats:
            break
        # extract geom and attributes to convert json to geodataframe    
        for feat in feats:
            attrs = feat.get("attributes", {})
            esri_geom = feat.get("geometry")
            if not esri_geom:
                continue

            try:
                geom = Geometry(esri_geom, sr=27700).as_shapely
                geom = make_valid(geom)  # repair invalid polygons
                if geom.is_empty:
                    continue
            except Exception as e:
                print("Skipping invalid geometry:", e)
                continue

            records.append(attrs)
            geoms.append(geom)

        offset += page_size

    print(f"Downloaded {len(records)} valid features.")


    # Create GeoDataFrame
    gdf = gpd.GeoDataFrame(records, geometry=geoms, crs="EPSG:27700")

    # Create folder if not already there for permanent data storage
    if not os.path.exists(f"{download_location}Original/{source}/{dataset}"):
        os.makedirs(f"{download_location}Original/{source}/{dataset}")
    if not os.path.exists(f"{download_location}Wiltshire/{source}/{dataset}"):
        os.makedirs(f"{download_location}Wiltshire/{source}/{dataset}")

    # Remove previous file
    if os.path.exists(f"{download_location}Original/{source}/{dataset}/{dataset}.gpkg"):
        os.remove(f"{download_location}Original/{source}/{dataset}/{dataset}.gpkg")

    # Write out original file
    gdf.to_file(f"{download_location}Original/{source}/{dataset}/{dataset}.gpkg", layer=f"{dataset}", driver="GPKG")
    
    # Clip to Wilts and write out
    if os.path.exists(f"{download_location}Wiltshire/{source}/{dataset}/{dataset}.gpkg"):
        os.remove(f"{download_location}Wiltshire/{source}/{dataset}/{dataset}.gpkg")
    gdf_wilts = gpd.clip(gdf, boundary_gdf)
    if gdf_wilts.empty == False:
        gdf_wilts.to_file(f"{download_location}Wiltshire/{source}/{dataset}/{dataset}.gpkg",layer=f"{dataset}", driver="GPKG")

    # Download dataset details
    try:
        response_details = requests.get(url_details)
        response_details.raise_for_status()
        details = response_details.json()
        date_last_edit = details['editingInfo']['dataLastEditDate']
        dt = datetime.fromtimestamp(date_last_edit / 1000, tz=timezone.utc)
        metadata = {"Date last edited":dt.strftime("%d/%m/%Y"),
                    "Date downloaded":date.today().strftime('%d/%m/%Y')} 
        with open(f"{download_location}Original/{source}/{dataset}/metadata.json", mode="w") as file:
            json.dump(metadata,file)
        with open(f"{download_location}Wiltshire/{source}/{dataset}/metadata.json", mode="w") as file:
            json.dump(metadata,file)
    except Exception as error:
        print(f"Request failed for details of {dataset}: {error}")

https://services.arcgis.com/JJzESW51TqeY9uat/arcgis/rest/services/Ramsar_England/FeatureServer/0/query?
Downloaded 1291 valid features.
https://services.arcgis.com/JJzESW51TqeY9uat/arcgis/rest/services/SSSI_England/FeatureServer/0/query?
Downloaded 2128 valid features.
https://services.arcgis.com/JJzESW51TqeY9uat/arcgis/rest/services/SSSI_Units_England/FeatureServer/0/query?
Downloaded 28305 valid features.
https://services.arcgis.com/JJzESW51TqeY9uat/arcgis/rest/services/Special_Areas_of_Conservation_England/FeatureServer/0/query?
Downloaded 250 valid features.
https://services.arcgis.com/JJzESW51TqeY9uat/arcgis/rest/services/Special_Protection_Areas_England/FeatureServer/0/query?
Downloaded 250 valid features.
https://services.arcgis.com/JJzESW51TqeY9uat/arcgis/rest/services/SSSI_Impact_Risk_Zones_England/FeatureServer/0/query?
Downloaded 104062 valid features.


DataSourceError: sqlite3_open(G:/GIS_Data/External/Open_Source/Wiltshire/NE/SSSI Impact Risk Zones (England)/SSSI Impact Risk Zones (England).gpkg) failed: unable to open database file